In [84]:
import os
from array import array
import numpy as np
from PIL import Image
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tqdm import tqdm

x_train = []
y_train = []

dayTime_train_path = "C:/Users/alice/dayTime_savedd/"
dayTime_train_fileDir = os.listdir(dayTime_train_path)
night_train_path = "C:/Users/alice/night_savedd/"
night_train_fileDir = os.listdir(night_train_path)

for file in dayTime_train_fileDir:
    imag = Image.open(dayTime_train_path + file) 
    data = np.array(imag)
    x_train.append(data)

for file in night_train_fileDir:
    imag = Image.open(night_train_path + file) 
    data = np.array(imag)
    y_train.append(data)

x_train = np.array(x_train)
x_train = x_train.reshape(98,784)
x_train = (x_train.astype(np.float32)-127.5)/127.5
y_train = np.array(y_train)
y_train = y_train.reshape(1,784)
y_train = (y_train.astype(np.float32)-127.5)/127.5

def build_generator():
    model = Sequential()
    model.add(Dense(units=1024,input_dim=784))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(units=512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(units=1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(units=784,activation='tanh'))
    model.compile(loss='binary_crossentropy',optimizer=Adam(0.0002,0.5))
    return model

def build_discriminator():
    model = Sequential()
    model.add(Dense(units=1024,input_dim=784))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3)) #防止過度擬合
    model.add(Dense(units=512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Dense(units=256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Dense(units=1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=Adam(0.0002,0.5))
    return model

def build_GAN(discriminator,generator):
    discriminator.trainable = False
    GAN_input = Input(shape = (784,))
    x = generator(GAN_input)
    GAN_output = discriminator(x)
    GAN = Model(inputs = GAN_input,outputs = GAN_output)
    GAN.compile(loss = 'binary_crossentropy',optimizer=Adam(0.0002,0.5))
    return GAN

def save_image(generator,epoch,example=25,dim=(5,5),figsize=(10,10)):
    generator_images = generator.predict(x_train)
    generator_images = generator_images.reshape(98,28,28)
    plt.figure(figsize=figsize)
    for i in range(generator_images.shape[0]):
        plt.subplot(dim[0],dim[1],i+1)
        plt.imshow(generator_images[i],interpolation='nearest',cmap='Greys')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('GAN_images %d,.png'%epoch)

def train_GAN(epochs=50, batch_size=256): 
    
    #Loading the data
    print(y_train)
    print(y_train.shape[1])
    generator=build_generator()
    discriminator=build_discriminator()
    GAN=build_GAN(discriminator, generator)
    
    for i in range(1, epochs+1):
        print("Epoch %d" %i)
        
        for _ in tqdm(range(batch_size)):   #每次都是128張照片訓練
            fake_images= generator.predict(y_train)
            
            real_images=x_train[np.random.randint(0,x_train.shape[0], batch_size)]
            
            label_fake = np.zeros(batch_size)
            label_real = np.ones(batch_size)
            
            x=np.concatenate([fake_images, real_images])
            y=np.concatenate([label_fake, label_real])
            
            discriminator.trainable=True
            discriminator.train_on_batch(x,y)
            
            discriminator.trainable=False
            GAN.train_on_batch(y_train, label_real)
        if i==1 or i%2==0:
            save_image(generator,i) 

train_GAN(epochs=20, batch_size=256)
    

        
        
print('suss')


[[-0.92156863 -0.9137255  -0.827451   -1.         -0.88235295 -0.8039216
  -1.         -0.7647059  -0.48235294 -0.40392157 -0.3882353  -0.44313726
  -0.45882353 -0.41960785 -0.44313726 -0.5137255  -0.5137255  -0.45882353
  -0.4509804  -0.38039216 -0.45882353 -0.5372549  -0.43529412 -0.49019608
  -0.84313726 -0.78039217 -0.75686276 -0.88235295 -0.9607843  -0.9529412
  -0.88235295 -1.         -0.8980392  -0.81960785 -1.         -0.79607844
  -0.5137255  -0.43529412 -0.39607844 -0.4117647  -0.41960785 -0.40392157
  -0.4117647  -0.4509804  -0.5137255  -0.39607844 -0.43529412 -0.42745098
  -0.44313726 -0.45882353 -0.39607844 -0.49019608 -0.654902   -0.64705884
  -0.6156863  -0.7647059  -0.9607843  -0.96862745 -0.90588236 -1.
  -0.8745098  -0.79607844 -0.9764706  -0.7882353  -0.4745098  -0.52156866
  -0.5529412  -0.49019608 -0.3882353  -0.34117648 -0.3882353  -0.46666667
  -0.5529412  -0.3882353  -0.49019608 -0.5137255  -0.44313726 -0.42745098
  -0.41960785 -0.5372549  -0.54509807 -0.5843137

  0%|                                                                                          | 0/256 [00:00<?, ?it/s]


ValueError: in user code:

    C:\Users\alice\anaconda3\envs\testAI\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\alice\anaconda3\envs\testAI\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\alice\anaconda3\envs\testAI\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\alice\anaconda3\envs\testAI\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\alice\anaconda3\envs\testAI\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\alice\anaconda3\envs\testAI\lib\sit